In [2]:
#import modules and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsb
import requests
#from bs4 import BeautifulSoup
from scipy.stats import skew, kurtosis
import csv

Load database

In [3]:
# create dataframe using data in the file
pdb =  pd.read_csv('core_database/destress_data_pdb.csv')
pdb.columns = ['design_name', 'file_name', 'full_sequence', 'dssp_assignment',
                'composition_ALA', 'composition_CYS', 'composition_ASP', 'composition_GLU', 
                'composition_PHE', 'composition_GLY', 'composition_HIS', 'composition_ILE', 
                'composition_LYS', 'composition_LEU', 'composition_MET', 'composition_ASN', 
                'composition_PRO', 'composition_GLN', 'composition_ARG', 'composition_SER', 
                'composition_THR', 'composition_VAL', 'composition_TRP', 'composition_UNK', 
                'composition_TYR', 'ss_prop_alpha_helix', 'ss_prop_beta_bridge', 'ss_prop_beta_strand', 
                'ss_prop_3_10_helix', 'ss_prop_pi_helix', 'ss_prop_hbonded_turn', 'ss_prop_bend', 
                'ss_prop_loop', 'hydrophobic_fitness', 'isoelectric_point', 'charge', 'mass', 
                'num_residues', 'packing_density', 'budeff_total', 'budeff_steric', 'budeff_desolvation', 
                'budeff_charge', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 
                'evoef2_interS_total', 'evoef2_interD_total', 'dfire2_total', 'rosetta_total', 
                'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 
                'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
                'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 
                'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 
                'rosetta_omega', 'rosetta_pro_close', 'rosetta_yhh_planarity', 'aggrescan3d_total_value', 
                'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value']

# create dataframe for the culling data set using data in the file
pisces = pd.read_csv('core_database/cullpdb_pc95.0_res0.0-3.0_len40-10000_R0.3_Xray_d2023_02_13_chains47410.csv')
pisces.columns = ["PDBchain", "len", "method", "resol", "rfac", "freerfac"]

In [ ]:
#convert Piscesculled data from txt to csv
# Read the text file into a pandas DataFrame
#df = pd.read_csv("core_database/cullpdb_pc95.0_res0.0-3.0_len40-10000_R0.3_Xray_d2023_02_13_chains47410.txt", sep='\s+', header=0)
# Assign column names to the DataFrame
#df.columns = ["PDBchain", "len", "method", "resol", "rfac", "freerfac"]
# Save the DataFrame as a CSV file
#df.to_csv("cullpdb_pc95.0_res0.0-3.0_len40-10000_R0.3_Xray_d2023_02_13_chains47410.csv", index=False)


Normalize the energy function data by number of residues

In [4]:
pdb.loc[:,[
    'hydrophobic_fitness', 'budeff_total', 'budeff_steric', 'budeff_desolvation', 
                'budeff_charge', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 
                'evoef2_interS_total', 'evoef2_interD_total', 'dfire2_total', 'rosetta_total', 
                'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 
                'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
                'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 
                'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 
                'rosetta_omega', 'rosetta_pro_close', 'rosetta_yhh_planarity', 'aggrescan3d_total_value', 
                'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value'
],] = pdb.loc[:,[
    'hydrophobic_fitness', 'budeff_total', 'budeff_steric', 'budeff_desolvation', 
                'budeff_charge', 'evoef2_total', 'evoef2_ref_total', 'evoef2_intraR_total', 
                'evoef2_interS_total', 'evoef2_interD_total', 'dfire2_total', 'rosetta_total', 
                'rosetta_fa_atr', 'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 
                'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 
                'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc', 
                'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun', 
                'rosetta_omega', 'rosetta_pro_close', 'rosetta_yhh_planarity', 'aggrescan3d_total_value', 
                'aggrescan3d_avg_value', 'aggrescan3d_min_value', 'aggrescan3d_max_value'
],].div(pdb['num_residues'],axis=0) 

Filter the de-stress data by Pisces

In [5]:
#generate a list of the culling protein id PDBchain_list
PDBchain = pisces['PDBchain'].str.extract(r'(\w{4})').squeeze()
PDBchain_list = PDBchain.tolist()

#create a new DataFrame pdb_cull for the data in PDB database which are also present in the cull_list from Pieces
pdb1 = pdb.copy() # a imaginary pdb data set
pdb1['design_name_extracted'] = pdb['design_name'].str[-4:].str.upper()
mask = pdb1['design_name_extracted'].isin(PDBchain_list)
pdb_cull = pdb1[mask]
pdb_cull = pdb_cull.drop(['design_name_extracted'],axis=1)
pdb_cull.reset_index(drop=True, inplace=True)
pdb_cull.to_csv('core_database/destress_data_culled_pdb.csv')

In [6]:
pdb_cull

,design_name,file_name,full_sequence,dssp_assignment,composition_ALA,composition_CYS,composition_ASP,composition_GLU,composition_PHE,composition_GLY,...,rosetta_rama_prepro,rosetta_p_aa_pp,rosetta_fa_dun,rosetta_omega,rosetta_pro_close,rosetta_yhh_planarity,aggrescan3d_total_value,aggrescan3d_avg_value,aggrescan3d_min_value,aggrescan3d_max_value
0,pdb1vbi,/input_path/pdb1vbi.ent,MRWRADFLSAWAEALLRKAGADEPSAKAVAWALVEADLRGVGSHGL...,-EE-HHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHHTT-GGGSG...,0.147059,0.000000,0.038235,0.085294,0.032353,0.117647,...,0.140494,-0.165805,1.633250,-0.012019,0.086942,0.0,-0.692081,-0.002036,-0.011772,0.009778
1,pdb3jte,/input_path/pdb3jte.ent,LAKILVIDDESTILQNIKFLLEIDGNEVLTASSSTEGLRIFTENCN...,--EEEEE-S-HHHHHHHHHHHHHTT-EEEEESSHHHHHHHHHHTTT...,0.071429,0.007937,0.071429,0.063492,0.023810,0.047619,...,-0.051496,-0.169816,2.067319,-0.010775,0.022928,0.0,-0.632446,-0.005019,-0.020245,0.010692
2,pdb5xmg,/input_path/pdb5xmg.ent,YQECLGRTFEIPEEEWATYDASRVWQISKGGGHNFTAEVTAVGDNG...,-EE--B--EE-S---B-BB-GGGGGGGGG----BSSSSSB-TT--E...,0.067647,0.005882,0.067647,0.073529,0.047059,0.064706,...,0.402435,-0.086517,3.867529,0.468314,0.364867,0.0,-1.118997,-0.003291,-0.011925,0.004388
3,pdb7va8,/input_path/pdb7va8.ent,SCPHVALLLSSGMGHLTPCLRFAATLVQHHCRVTIITNYPTVSVAE...,---EEEEE--SSHHHHHHHHHHHHHHHTTT-EEEEEE-SS-SSHHH...,0.055866,0.018994,0.042458,0.067039,0.050279,0.071508,...,0.146166,-0.110057,3.932635,0.183460,0.136996,0.0,-0.550916,-0.000616,-0.004521,0.001800
4,pdb4ewt,/input_path/pdb4ewt.ent,MNQQLIETLKSKEGKMIEIRRYLHQHPELSFHEDETAKYIAEFYKG...,-HHHHHHHHHHTHHHHHHHHHHHHHS--BTT--HHHHHHHHHHTTT...,0.077821,0.007782,0.056420,0.083658,0.040856,0.089494,...,0.133969,-0.126761,2.892109,0.245879,0.251785,0.0,-0.719611,-0.000467,-0.002278,0.000928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41803,pdb3i05,/input_path/pdb3i05.ent,NYDKLIKDFGSHAIDEALLERIERVLGKKPHHFLRRGIFFSHRDLN...,-HHHHHHHTT-EE--HHHHHHHHHHHTS---HHHHTTSEEEEESHH...,0.060698,0.012140,0.068285,0.059181,0.084977,0.053111,...,0.248860,-0.084868,3.316119,0.387947,0.228010,0.0,-0.699390,-0.001061,-0.005447,0.002235
41804,pdb1p2f,/input_path/pdb1p2f.ent,WKIAVVDDDKNILKKVSEKLQQLGRVKTFLTGEDFLNDEEAFHVVV...,-EEEEE-S-HHHHHHHHHHHTTTSEEEEESSHHHHHH--S--SEEE...,0.042453,0.004717,0.084906,0.089623,0.061321,0.061321,...,0.114032,-0.151796,1.949128,0.115300,0.186911,0.0,-1.128883,-0.005325,-0.019070,0.009686
41805,pdb5te9,/input_path/pdb5te9.ent,DLIHILLVEDSPTDVMITREAFDYYKLLNPLHVAGDGVAAMEFLRR...,---EEEEE---HHHHHHHHHHHHHTT--S-EEEESSHHHHHHHHHT...,0.058394,0.007299,0.087591,0.065693,0.043796,0.051095,...,0.022112,-0.145350,3.291107,0.237111,0.154357,0.0,-0.497196,-0.003629,-0.026777,0.021384
41806,pdb5joe,/input_path/pdb5joe.ent,MGPVTLIKDIENQTVLKDNDAVFEIDIKINYPEIKLSWYKGTEKLE...,--SEEEEE----EEEETTS-EEEEEEEEE-STT---EEEETTEE--...,0.032609,0.021739,0.086957,0.076087,0.021739,0.054348,...,0.065833,-0.158601,1.739008,0.123959,0.076388,0.0,-0.898161,-0.009763,-0.036198,0.020502


Clean the database

In [7]:
#drop duplicated sequences
pdb_cull = pdb_cull.drop_duplicates(subset='full_sequence', keep='first')
pdb_cull.dropna(subset=['full_sequence'], inplace=True)
pdb_cull.reset_index(drop=True, inplace=True)
pdb_cull

/var/folders/jq/ph87kw_d695bbwx2yc22bgxh0000gn/T/ipykernel_8800/2599579283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdb_cull.dropna(subset=['full_sequence'], inplace=True)


,design_name,file_name,full_sequence,dssp_assignment,composition_ALA,composition_CYS,composition_ASP,composition_GLU,composition_PHE,composition_GLY,...,rosetta_rama_prepro,rosetta_p_aa_pp,rosetta_fa_dun,rosetta_omega,rosetta_pro_close,rosetta_yhh_planarity,aggrescan3d_total_value,aggrescan3d_avg_value,aggrescan3d_min_value,aggrescan3d_max_value
0,pdb1vbi,/input_path/pdb1vbi.ent,MRWRADFLSAWAEALLRKAGADEPSAKAVAWALVEADLRGVGSHGL...,-EE-HHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHHTT-GGGSG...,0.147059,0.000000,0.038235,0.085294,0.032353,0.117647,...,0.140494,-0.165805,1.633250,-0.012019,0.086942,0.0,-0.692081,-0.002036,-0.011772,0.009778
1,pdb3jte,/input_path/pdb3jte.ent,LAKILVIDDESTILQNIKFLLEIDGNEVLTASSSTEGLRIFTENCN...,--EEEEE-S-HHHHHHHHHHHHHTT-EEEEESSHHHHHHHHHHTTT...,0.071429,0.007937,0.071429,0.063492,0.023810,0.047619,...,-0.051496,-0.169816,2.067319,-0.010775,0.022928,0.0,-0.632446,-0.005019,-0.020245,0.010692
2,pdb5xmg,/input_path/pdb5xmg.ent,YQECLGRTFEIPEEEWATYDASRVWQISKGGGHNFTAEVTAVGDNG...,-EE--B--EE-S---B-BB-GGGGGGGGG----BSSSSSB-TT--E...,0.067647,0.005882,0.067647,0.073529,0.047059,0.064706,...,0.402435,-0.086517,3.867529,0.468314,0.364867,0.0,-1.118997,-0.003291,-0.011925,0.004388
3,pdb7va8,/input_path/pdb7va8.ent,SCPHVALLLSSGMGHLTPCLRFAATLVQHHCRVTIITNYPTVSVAE...,---EEEEE--SSHHHHHHHHHHHHHHHTTT-EEEEEE-SS-SSHHH...,0.055866,0.018994,0.042458,0.067039,0.050279,0.071508,...,0.146166,-0.110057,3.932635,0.183460,0.136996,0.0,-0.550916,-0.000616,-0.004521,0.001800
4,pdb4ewt,/input_path/pdb4ewt.ent,MNQQLIETLKSKEGKMIEIRRYLHQHPELSFHEDETAKYIAEFYKG...,-HHHHHHHHHHTHHHHHHHHHHHHHS--BTT--HHHHHHHHHHTTT...,0.077821,0.007782,0.056420,0.083658,0.040856,0.089494,...,0.133969,-0.126761,2.892109,0.245879,0.251785,0.0,-0.719611,-0.000467,-0.002278,0.000928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39883,pdb3i05,/input_path/pdb3i05.ent,NYDKLIKDFGSHAIDEALLERIERVLGKKPHHFLRRGIFFSHRDLN...,-HHHHHHHTT-EE--HHHHHHHHHHHTS---HHHHTTSEEEEESHH...,0.060698,0.012140,0.068285,0.059181,0.084977,0.053111,...,0.248860,-0.084868,3.316119,0.387947,0.228010,0.0,-0.699390,-0.001061,-0.005447,0.002235
39884,pdb1p2f,/input_path/pdb1p2f.ent,WKIAVVDDDKNILKKVSEKLQQLGRVKTFLTGEDFLNDEEAFHVVV...,-EEEEE-S-HHHHHHHHHHHTTTSEEEEESSHHHHHH--S--SEEE...,0.042453,0.004717,0.084906,0.089623,0.061321,0.061321,...,0.114032,-0.151796,1.949128,0.115300,0.186911,0.0,-1.128883,-0.005325,-0.019070,0.009686
39885,pdb5te9,/input_path/pdb5te9.ent,DLIHILLVEDSPTDVMITREAFDYYKLLNPLHVAGDGVAAMEFLRR...,---EEEEE---HHHHHHHHHHHHHTT--S-EEEESSHHHHHHHHHT...,0.058394,0.007299,0.087591,0.065693,0.043796,0.051095,...,0.022112,-0.145350,3.291107,0.237111,0.154357,0.0,-0.497196,-0.003629,-0.026777,0.021384
39886,pdb5joe,/input_path/pdb5joe.ent,MGPVTLIKDIENQTVLKDNDAVFEIDIKINYPEIKLSWYKGTEKLE...,--SEEEEE----EEEETTS-EEEEEEEEE-STT---EEEETTEE--...,0.032609,0.021739,0.086957,0.076087,0.021739,0.054348,...,0.065833,-0.158601,1.739008,0.123959,0.076388,0.0,-0.898161,-0.009763,-0.036198,0.020502


In [6]:
#decide how to drop the rows with missing values
#missing_no_incull containing the number of missing values in each index in the pdb_culled
missing_no_incull = pdb_cull.isnull().sum()
missing_no_incull

#按照missing_no_incull中的value来分组
the_clusters =missing_no_incull.groupby(missing_no_incull)

#print the groups
for name, group in the_clusters:
    print("Name:", name)
    print ('Group',group)
group_10 = the_clusters.get_group(10)
print(group_10.index.tolist())

Name: 0
Group design_name             0
file_name               0
full_sequence           0
dssp_assignment         0
composition_ALA         0
composition_CYS         0
composition_ASP         0
composition_GLU         0
composition_PHE         0
composition_GLY         0
composition_HIS         0
composition_ILE         0
composition_LYS         0
composition_LEU         0
composition_MET         0
composition_ASN         0
composition_PRO         0
composition_GLN         0
composition_ARG         0
composition_SER         0
composition_THR         0
composition_VAL         0
composition_TRP         0
composition_UNK         0
composition_TYR         0
ss_prop_alpha_helix     0
ss_prop_beta_bridge     0
ss_prop_beta_strand     0
ss_prop_3_10_helix      0
ss_prop_pi_helix        0
ss_prop_hbonded_turn    0
ss_prop_bend            0
ss_prop_loop            0
isoelectric_point       0
charge                  0
mass                    0
num_residues            0
packing_density         

In [16]:

missing_valuesin_budeff_total = pdb_cull[pdb_cull['dfire2_total'].isnull()]

missing_valuesin_budeff_total.sample(n=5,random_state=1)

,design_name,file_name,full_sequence,dssp_assignment,composition_ALA,composition_CYS,composition_ASP,composition_GLU,composition_PHE,composition_GLY,...,rosetta_rama_prepro,rosetta_p_aa_pp,rosetta_fa_dun,rosetta_omega,rosetta_pro_close,rosetta_yhh_planarity,aggrescan3d_total_value,aggrescan3d_avg_value,aggrescan3d_min_value,aggrescan3d_max_value
5794,pdb7pje,/input_path/pdb7pje.ent,REVISIHVGQGGIQVGNACWELFCLEHGIQPDGQMPSDAFNTFFSE...,--EEEEEEHHHHHHHHHHHHHHHHHHHT--TTS------GGGTEEE...,0.081192,0.018499,0.060637,0.066804,0.043165,0.082220,...,0.066451,-0.117472,2.297080,0.200002,0.099718,0.003217,-0.675430,-0.000347,-0.001899,0.001330
36160,pdb2q8b,/input_path/pdb2q8b.ent,YMGNPWTEYMAKYDIEEVHGSGIRVDLGEDAEVAGTQYRLPSGKCP...,----TTHHHHGGG-HHHHT-S-SSS---EEEEETTEEEEE---SS-...,0.051136,0.026989,0.053977,0.051136,0.041193,0.062500,...,0.171097,-0.149230,2.382614,0.206672,0.066372,0.000000,-0.621091,-0.000882,-0.004803,0.002467
2050,pdb5my6,/input_path/pdb5my6.ent,QVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNA...,-EEE-B--TT---S-HHHHHHHHHHHHTT--EEESBEEEES--TT-...,0.058122,0.071535,0.038748,0.058122,0.029806,0.086438,...,0.112490,-0.182512,2.188100,0.286930,0.168155,0.000000,-0.590863,-0.000881,-0.005165,0.002835
940,pdb4jyl,/input_path/pdb4jyl.ent,NLYFQSLVEIERRGDASLIVLSRPEKLNAINLELADLADQFSKAEK...,--TT---EEEEEETTEEEEEE--GGGTT-B----HHHHHHHHHHHH...,0.126850,0.004228,0.076110,0.063425,0.032417,0.076110,...,0.298189,-0.133497,2.397883,0.881132,0.075210,0.000000,-0.671623,-0.000473,-0.002520,0.001983
676,pdb2e8y,/input_path/pdb2e8y.ent,MVSIRRSFEAYVDDMNIITVLIPAEQKEIMTPPFRLETEITDFPLA...,------SSEEEEEETTEEEEEEEGGGTTTS-S-EEEEETTEEEEEE...,0.080056,0.011236,0.064607,0.067416,0.049157,0.063202,...,0.063554,-0.169647,2.230298,-0.010592,0.114483,0.000000,-0.643688,-0.000452,-0.002653,0.001573


In [15]:
# Merge the two DataFrames on the 'Entry' column and use the 'inner' join method to find the rows that are in both DataFrames
overlap_df = pd.merge(missing_valuesin_budeff_total, missing_valuesin_aggrescan3d_total_value, on='design_name', how='inner')

# Calculate the number of overlapping rows
num_overlap = len(overlap_df)

# Calculate the number of unique rows in each DataFrame
num_unique_evoef2 = len(missing_valuesin_budeff_total) - num_overlap
num_unique_aggrescan3d = len(missing_valuesin_aggrescan3d_total_value) - num_overlap

print(f"Number of overlapping rows: {num_overlap}")
print(f"Number of unique rows in 'missing_valuesin_evoef2_intraR_total': {num_unique_evoef2}")
print(f"Number of unique rows in 'missing_valuesin_aggrescan3d_total_value': {num_unique_aggrescan3d}")
# Merge the two DataFrames on the 'Entry' column and use the 'outer' join method
merged_df = pd.merge(missing_valuesin_aggrescan3d_total_value, missing_valuesin_budeff_total, on='design_name', how='outer', indicator=True)

# Filter the merged DataFrame to select the rows unique to 'missing_valuesin_aggrescan3d_total_value'
unique_rows_aggrescan3d = merged_df.loc[merged_df['_merge'] == 'right_only']

# Drop the '_merge' column as it is not needed anymore
unique_rows_aggrescan3d.drop(columns=['_merge'], inplace=True)

print("Unique rows in 'missing_valuesin_aggrescan3d_total_value':")
print(unique_rows_aggrescan3d.sample(n=5, random_state=1))


Number of overlapping rows: 716
Number of unique rows in 'missing_valuesin_evoef2_intraR_total': 2977
Number of unique rows in 'missing_valuesin_aggrescan3d_total_value': 289
Unique rows in 'missing_valuesin_aggrescan3d_total_value':
     design_name file_name_x full_sequence_x dssp_assignment_x  \
1450     pdb7re6         NaN             NaN               NaN   
3692     pdb7aah         NaN             NaN               NaN   
2546     pdb4kpi         NaN             NaN               NaN   
1526     pdb5z3e         NaN             NaN               NaN   
1372     pdb7bbv         NaN             NaN               NaN   

      composition_ALA_x  composition_CYS_x  composition_ASP_x  \
1450                NaN                NaN                NaN   
3692                NaN                NaN                NaN   
2546                NaN                NaN                NaN   
1526                NaN                NaN                NaN   
1372                NaN                NaN  

/var/folders/jq/ph87kw_d695bbwx2yc22bgxh0000gn/T/ipykernel_20274/2739140378.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_rows_aggrescan3d.drop(columns=['_merge'], inplace=True)


In [ ]:
#plot the missing value in the pisces database 
fig = plt.figure(figsize=(24, 8))
missing_no_incull.plot(kind='bar')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Bar Chart of Missing Values in the culled database')
plt.show()

In [ ]:
# Generate Bar Chart of Values in the pdb database
missing_valuess_pdb = pdb.isnull().sum()
#print(missing_values[missing_values > 0])

#plot the number of missing values under each index
fig = plt.figure(figsize=(24, 8))
missing_valuess_pdb.plot(kind='bar')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Bar Chart of Missing Values in PDB')
plt.show()

In [ ]:
pdb_No_NaN = pdb.dropna() #remove all rows containing NaN
nan_rows = pdb[pdb.isna().any(axis=1)] #series that containg the NaN value

# Assuming that the dataframe is called pdb
aggrescan3d_avg_value = pdb['aggrescan3d_avg_value']
non_aggrescan3d_avg_value = pdb_No_NaN['aggrescan3d_avg_value']

# Removing NaN values
aggrescan3d_avg_value = aggrescan3d_avg_value.dropna()

# Sampling 10,000 values
sample = aggrescan3d_avg_value.sample(n=10000, random_state=0)
non_sample = non_aggrescan3d_avg_value.sample(n=10000, random_state=0)
print (sample.describe(), non_sample.describe(),pdb['aggrescan3d_avg_value'].describe(), pdb_No_NaN['aggrescan3d_avg_value'].describe())
#print (pdb['aggrescan3d_avg_value'].describe(), pdb_No_NaN['aggrescan3d_avg_value'].describe())
#nan_rows.count()

In [ ]:
sample

In [ ]:
#Compare the summary statistics between the pdb database: 1. remove dfire2 column at first, then remove all rows containing NaN 2. remove all NaN value altogether

#1
pdb_nodfire = pdb_cull.drop(['dfire2_total'],axis=1)
pdb_nodfire = pdb_nodfire.dropna(how='any')
#print(pdb_nodfire.shape)
#we have reduced the size of dataframe from 39,888 to  35,895


#2
pdb_withdfire = pdb_cull.dropna(how='any')
#print (pdb_withdfire.shape)
#we have reduced the size of dataframe from 39,888 to 24113


# compare the summary statistics between pdb_nodfire and pdb_withdfire
sample_nod = pdb_nodfire.sample(n=1000, random_state=0)
sample_yesd = pdb_withdfire.sample(n=1000, random_state=0)
#print (sample_nod.describe(), sample_yesd.describe())
sample = pd.concat([sample_nod.describe(),sample_yesd.describe()])
sample = sample[['hydrophobic_fitness','isoelectric_point','mass','packing_density',
                'budeff_total','evoef2_total','rosetta_total']]
sample.to_csv('./Output tables/descriptive_statistics_comparison_with_or_without_dfire.csv',index=False)

In [ ]:
pdb_nodfire

In [ ]:
pdb_nodfire.to_csv('./core_database/destress_data_cleaned_pdb_nodfire.csv.csv',index=False)

In [ ]:
#Cluster the data about the amino acid composition into subgroups
pdb_nodfire['composition_hydrophobic'] =pdb_nodfire['composition_ALA']+ pdb_nodfire['composition_VAL'] + pdb_nodfire['composition_LEU'] + pdb_nodfire['composition_ILE'] + pdb_nodfire['composition_MET'] + pdb_nodfire['composition_PHE'] + pdb_nodfire['composition_TYR'] + pdb_nodfire['composition_TRP']
pdb_nodfire['composition_polar'] = pdb_nodfire['composition_ASN'] + pdb_nodfire['composition_GLN'] + pdb_nodfire['composition_SER'] + pdb_nodfire['composition_THR']
pdb_nodfire['composition_acidic'] = pdb_nodfire['composition_ASP'] + pdb_nodfire['composition_GLU'] 
pdb_nodfire['composition_basic'] = pdb_nodfire['composition_ARG'] + pdb_nodfire['composition_HIS'] + pdb_nodfire['composition_LYS']


In [ ]:
a = pdb_nodfire[['ss_prop_alpha_helix', 'ss_prop_beta_bridge', 'ss_prop_beta_strand',
       'ss_prop_3_10_helix', 'ss_prop_pi_helix', 'ss_prop_hbonded_turn',
       'ss_prop_bend', 'ss_prop_loop', 'hydrophobic_fitness',
       'isoelectric_point', 'num_residues',
       'packing_density', 'budeff_total', 'budeff_steric',
       'budeff_desolvation', 'budeff_charge', 'evoef2_total',
        'evoef2_intraR_total', 'evoef2_interS_total',
        'rosetta_total', 'rosetta_fa_atr',
       'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec',
       'rosetta_fa_sol', 'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4',
       'rosetta_hbond_lr_bb', 'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc',
       'rosetta_hbond_sc',  'rosetta_rama_prepro',
       'rosetta_p_aa_pp', 'rosetta_fa_dun', 'rosetta_omega',
       'rosetta_pro_close',  
       'aggrescan3d_avg_value', 'aggrescan3d_min_value',
       'aggrescan3d_max_value']]
b=a.describe()
b.to_csv('PDB_nodfire_descriptive_statistics.csv')



In [ ]:
# calculate the correlation between columns using the .corr() method
corr_matrix = pdb_culled.corr()

# print the correlation matrix
print(corr_matrix)
corr_matrix.to_csv("correlation_between_each_index_in_the_culled_pdb.csv")

plt.hist(pdb_cull['aggrescan3d_total_value'], bins=30)
plt.xlabel('Data values')
plt.ylabel('Frequency')
plt.title('Histogram of Data')
plt.show()

In [ ]:
#infer the distribution of data
skewness =pdb_nodfire .skew()
kurt = pdb_nodfire.kurtosis()
skewness
#distribution = pd.concat([skewness, kurt],axis=1)
#distribution.to_csv('distribution_pdb_nodfire.csv')


In [ ]:
pdb_nodfire_num = pdb_nodfire.drop(['design_name', 'file_name', 'full_sequence', 'dssp_assignment','composition_UNK','rosetta_yhh_planarity'],axis=1)
for col in pdb_nodfire_num:
    pdb_nodfire_num[col] = (pdb_nodfire_num[col] - pdb_nodfire_num[col].min()) / (pdb_nodfire_num[col].max() - pdb_nodfire_num[col].min())
pdb_nodfire_matrix = pdb_nodfire_num.cov ()
pdb_nodfire_matrix.to_csv('covariance_matrix_in_culled_pdb_without_dfire2.csv')


In [ ]:
pdb_sample = pdb_nodfire[['design_name', 'hydrophobic_fitness', 'isoelectric_point',
                    'charge', 'mass', 'num_residues', 'packing_density', 'budeff_total',
                    'evoef2_total','evoef2_intraR_total',
                    'rosetta_total', 'rosetta_fa_atr',
                    'rosetta_fa_rep', 'rosetta_fa_intra_rep', 'rosetta_fa_elec', 'rosetta_fa_sol',
                    'rosetta_lk_ball_wtd', 'rosetta_fa_intra_sol_xover4', 'rosetta_hbond_lr_bb',
                    'rosetta_hbond_sr_bb', 'rosetta_hbond_bb_sc', 'rosetta_hbond_sc',
                    'rosetta_dslf_fa13', 'rosetta_rama_prepro', 'rosetta_p_aa_pp', 'rosetta_fa_dun',
                    'rosetta_omega', 'rosetta_pro_close', 'rosetta_yhh_planarity',
                    'aggrescan3d_total_value', 'aggrescan3d_avg_value']]
pdb_sample.shape

In [ ]:
pdb_nodfire.columns.to_list

In [ ]:

pdb_nodfire_matrix = pd.read_csv ('covariance_matrix_in_culled_pdb_without_dfire2.csv')
pdb_nodfire_matrix.head

In [ ]:
pdb_nodfire.to_csv('pdb_noNaN_nodfire.csv')
